# Caso practico : Arboles de Decision

en este caso de uso practico se pretende resolver un problema de deteccion de maldware en dispositivos android mediante el analasis del trafico de red que genera el dispositivo mediante el uso de arboles de desicion. 


Android adware and general malware dataset (CIC-AAGM2017)

The sophisticated and advanced Android malware is able to identify the presence of the emulator used by the malware analyst and in response, alter its behaviour to evade detection. To overcome this issue, we installed the Android applications on the real device and captured its network traffic. 

CICAAGM dataset is captured by installing the Android apps on the real smartphones semi-automated. The dataset is generated from 1,900 applications with the following three categories:

**1. Adware (250 apps)**

    Airpush: Designed to deliver unsolicited advertisements to the user’s systems for information stealing.

    Dowgin: Designed as an advertisement library that can also steal the user’s information.

    Kemoge: Designed to take over a user’s Android device. This adware is a hybrid of botnet and disguises itself as popular apps via repackaging.

    Mobidash: Designed to display ads and to compromise user’s personal information.

    Shuanet: Similar to Kemoge, Shuanet is also designed to take over a user’s device.

**2. General Malware (150 apps)**

    AVpass: Designed to be distributed in the guise of a Clock app.

    FakeAV: Designed as a scam that tricks user to purchase a full version of the software in order to re-mediate non-existing infections.

    FakeFlash/FakePlayer: Designed as a fake Flash app in order to direct users to a website (after successfully installed).

    GGtracker: Designed for SMS fraud (sends SMS messages to a premium-rate number) and information stealing.

    Penetho: Designed as a fake service (hacktool for Android devices that can be used to crack the WiFi password). The malware is also able to infect the user’s computer via infected email attachment, fake updates, external media and infected documents.

**3. Benign (1,500 apps)**

    2015 GooglePlay market (top free popular and top free new)

    2016 GooglePlay market (top free popular and top free new)

    License

The CICAAGM dataset consists of the following items is publicly available for researchers.

    .pcap files – the network traffic of both the malware and benign (20% malware and 80% benign)

    .csv files - the list of extracted network traffic features generated by the CIC-flowmeter

If you are using our dataset, you should cite our related paper that outlines the details of the dataset and its underlying principles:

    Arash Habibi Lashkari, Andi Fitriah A. Kadir, Hugo Gonzalez, Kenneth Fon Mbah and Ali A. Ghorbani, “Towards a Network-Based Framework for Android Malware Detection and Characterization”, In the proceeding of the 15th International Conference on Privacy, Security and Trust, PST, Calgary, Canada, 2017.


(Descargar DataSet:https://www.unb.ca/cic/datasets/android-adware.html)


In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, recall_score, precision_score, f1_score
from pandas import DataFrame

In [4]:
### Funciones Auxiliares

# crear una funcion que realice el particionado
def train_val_test_split(df, rstate = 42, shuffle = True, stratify = None):
    strat = df[stratify] if stratify else None
    train_set, test_set = train_test_split(
        df, test_size = 0.4, random_state =rstate, shuffle=shuffle, stratify=strat)
    strat = test_set[stratify] if stratify else  None
    val_set, test_set = train_test_split(
        test_set, test_size = 0.5, random_state = rstate, shuffle=shuffle, stratify=strat)
    return (train_set, val_set, test_set)

In [6]:
def remove_labels(df, label_name):
    X = df.drop(label_name, axis=1)
    y = df[label_name].copy()
    return (X, y) 

In [10]:
def evalued_resul(y_pred, y, y_prep_pred, y_prep, metric):
    print(metric._name_, "WITHOUT preparation:", metric (y_pred, y, average= 'weight'))
    print(metric._name_, "WITH preparation:", metric (y_prep_pred, y_prep, average= 'weighted'))

## lectura del DataSet 

In [12]:
df = pd.read_csv('datasets/datasets/TotalFeatures-ISCXFlowMeter.csv')

## 1.- Visualizacion del DataSet

In [13]:
df.head(10)


,duration,total_fpackets,total_bpackets,total_fpktl,total_bpktl,min_fpktl,min_bpktl,max_fpktl,max_bpktl,mean_fpktl,...,mean_idle,max_idle,std_idle,FFNEPD,Init_Win_bytes_forward,Init_Win_bytes_backward,RRT_samples_clnt,Act_data_pkt_forward,min_seg_size_forward,calss
0,1020586,668,1641,35692,2276876,52,52,679,1390,53.431138,...,0.0,-1,0.0,2,4194240,1853440,1640,668,32,benign
1,80794,1,1,75,124,75,124,75,124,75.000000,...,0.0,-1,0.0,2,0,0,0,1,0,benign
2,998,3,0,187,0,52,-1,83,-1,62.333333,...,0.0,-1,0.0,4,101888,-1,0,3,32,benign
3,189868,9,9,1448,6200,52,52,706,1390,160.888889,...,0.0,-1,0.0,2,4194240,2722560,8,9,32,benign
4,110577,4,6,528,1422,52,52,331,1005,132.000000,...,0.0,-1,0.0,2,155136,31232,5,4,32,benign
5,261876,7,6,1618,882,52,52,730,477,231.142857,...,0.0,-1,0.0,2,4194240,926720,3,7,32,benign
6,14,2,0,104,0,52,-1,52,-1,52.000000,...,0.0,-1,0.0,3,5824,-1,0,2,32,benign
7,29675,1,1,71,213,71,213,71,213,71.000000,...,0.0,-1,0.0,2,0,0,0,1,0,benign
8,806635,4,0,239,0,52,-1,83,-1,59.750000,...,0.0,-1,0.0,5,107008,-1,0,4,32,benign
9,56620,3,2,1074,719,52,52,592,667,358.000000,...,0.0,-1,0.0,3,128512,10816,1,3,32,benign


In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 631955 entries, 0 to 631954
Data columns (total 80 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   duration                 631955 non-null  int64  
 1   total_fpackets           631955 non-null  int64  
 2   total_bpackets           631955 non-null  int64  
 3   total_fpktl              631955 non-null  int64  
 4   total_bpktl              631955 non-null  int64  
 5   min_fpktl                631955 non-null  int64  
 6   min_bpktl                631955 non-null  int64  
 7   max_fpktl                631955 non-null  int64  
 8   max_bpktl                631955 non-null  int64  
 9   mean_fpktl               631955 non-null  float64
 10  mean_bpktl               631955 non-null  float64
 11  std_fpktl                631955 non-null  float64
 12  std_bpktl                631955 non-null  float64
 13  total_fiat               631955 non-null  int64  
 14  tota

In [15]:
df['calss'].value_counts()

calss
benign            471597
asware            155613
GeneralMalware      4745
Name: count, dtype: int64

In [ ]:
#### Buscando Correlaciones